In [3]:
import numpy as np
import pandas as pd
import scipy.spatial.distance as dist
from sympy import *
from collections import defaultdict

In [2]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [63]:
class PageRank:
    def __init__(self, edge_list, init_val=None, rounding_digit = 3, n_iter = 10):
        self.edge_list = edge_list
        self.prepare_page_dict()
        self.rounding_digit = rounding_digit
        self.n_iter = n_iter
        if init_val:
            self.init_val = init_val
        else:
            self.init_val = [1/len(self.page_list) for x in self.page_list]
    
    def prepare_page_dict(self):
        self.page_source_dict = defaultdict(list)
        self.page_dest_dict = defaultdict(list)
        self.page_list = []
        for s,d in self.edge_list:
            self.page_source_dict[s].append(d)
            self.page_dest_dict[d].append(s)
            
            if s not in self.page_list:
                self.page_list.append(s)
            if d not in self.page_list:
                self.page_list.append(d)
        self.page_list.sort()
        
    def evaluate(self):
        print('Rank of different webpages:\n')
#         print(self.page_source_dict)
#         print(self.page_dest_dict)
#         print(self.page_list)
        for page in self.page_list:
            if self.page_dest_dict[page]:
                s = ''
                for dest in self.page_dest_dict[page]:
                    s += f'R({dest})/{len(self.page_source_dict[dest])} +'
                s = s[:-1]
                display(Symbol(f'R({page})={s}'))
        self.iteration_matrix = pd.DataFrame(index=self.page_list)
        
        vals = {x:y for x, y in zip(self.page_list, self.init_val)}
        print(f'\nInitial values:\n{vals}')
        
        self.iteration_matrix['Initial Values'] = self.init_val
        n_iteration = 1
        cur_val = vals
        rank_list = []
        prev_rank_list = []
        while n_iteration <= self.n_iter:
            print(f'\nIteration {n_iteration}:\n')
            for page in self.page_list: 
                if self.page_dest_dict[page]:
                    s = ''
                    rank = 0
                    for dest in self.page_dest_dict[page]:
                        s += f'{cur_val[dest]}/{len(self.page_source_dict[dest])} +'
                        rank+= cur_val[dest]/len(self.page_source_dict[dest])
                    s = s[:-1]
                    
                    display(Symbol(f'R({page})={s} = {round(rank, self.rounding_digit)}'))
                else:
                    rank = cur_val[page]
                    display(Symbol(f'R({page}) = {rank}'))
                rank = round(rank, self.rounding_digit)
                vals[page] = rank
                rank_list.append(rank)
            #print(rank_list)
            
            self.iteration_matrix[f'Iteration {n_iteration}'] = rank_list
            if prev_rank_list == rank_list:
                print('\n Converged')
                break
            n_iteration +=1
            
            prev_rank_list = rank_list
            rank_list = []
            cur_val = vals
            print('--------------------------------------------------------------')
        self.iteration_matrix['Page rank'] = self.iteration_matrix[self.iteration_matrix.columns[-1]]. \
                                                    rank(method='dense', ascending=False).astype(int)
        display(self.iteration_matrix)

In [64]:
edge_list = [('A', 'C'),('C', 'A'),('A','B'), ('C', 'B'), ('B','D'), ('C','D'), ('D','C')]
PageRank(edge_list).evaluate()

Rank of different webpages:



R(A)=R(C)/3 

R(B)=R(A)/2 +R(C)/3 

R(C)=R(A)/2 +R(D)/1 

R(D)=R(B)/1 +R(C)/3 

...

R(B)=0.093/2 +0.28/3  = 0.14

R(C)=0.093/2 +0.233/1  = 0.28

R(D)=0.14/1 +0.28/3  = 0.233


 Converged


,Initial Values,Iteration 1,Iteration 2,Iteration 3,Iteration 4,Iteration 5,Iteration 6,Iteration 7,Iteration 8,Iteration 9,Iteration 10,Page rank
A,0.25,0.083,0.097,0.090,0.093,0.092,0.093,0.093,0.093,0.093,0.093,4
B,0.25,0.125,0.145,0.135,0.140,0.138,0.139,0.139,0.140,0.140,0.140,3
C,0.25,0.291,0.271,0.280,0.275,0.278,0.278,0.279,0.279,0.280,0.280,1
D,0.25,0.222,0.235,0.228,0.232,0.231,0.232,0.232,0.233,0.233,0.233,2
